<h1><b>ĐỒ ÁN CUỐI KỲ NMKHDL</b></h1>
<h2><b>POKEDEX</b></h2>

Họ tên: Nguyễn Đăng Quang

MSSV: 18120527



# Giới thiệu sơ lược
##### Có thể bạn chưa biết, Pocket Monsters viết tắt là Pokemon, một series game cũng như anime đình đám, series game Pokemon phát triển bởi GameFreak hiện vẫn đang làm mưa làm gió trên các hệ máy Nitendo và trên thị trường game toàn cầu. Nhân dịp gen9 (Scarlet & Violet) của game vừa ra mắt, chúng ta sẽ điểm lại các pokemon từ những thế hệ trước và thông số cũng như đặc tính của chúng thông qua Pokedex. Ở đây, chúng ta sẽ sử dụng "https://pokemondb.net/pokedex/all" là trang web chứa dữ liệu Pokedex, hay nói cách khác là danh sách của toàn bộ Pokemon ĐÃ xuất hiện xuyên suốt series game này. Let's go!

# Install và import
- bs4 (Beautiful Soup): Một thư viện Python dùng để phân tích các tài liệu HTML và XML. Nó tạo ra một cây phân tích cú pháp cho các trang được phân tích cú pháp có thể được sử dụng để trích xuất dữ liệu từ HTML, rất hữu ích cho web scrapping.
- requests: Cho phép gửi các yêu cầu HTTP bằng Python.

In [ ]:
!pip install bs4
!pip install requests

In [ ]:
import bs4
import requests
import time
import random as ran
import sys
import pandas as pd

# Thu thập và xử lý dữ liệu
- Lưu ý: Có thể các Pokemon có rất nhiều dạng (vùng miền, mega evolution, mega X/mega Y, nguyên thủy (primal form), tấn công/phòng thủ,...), hình dạng Pokemon thay đổi nhưng id thì vẫn giữ nguyên, vì vậy sẽ xuất hiện trường hợp một id lặp lại nhiều lần, chúng ta chỉ xét hình dạng cơ bản của chúng nên chỉ cần lấy 1 id cho mỗi Pokemon thôi. 

In [ ]:
url = 'https://pokemondb.net/pokedex/all'

source = requests.get(url).text
soup = bs4.BeautifulSoup(source,'html.parser')

# Thu thập id của toàn bộ pokemon
pokemon_id = soup.findAll('span', {'class': 'infocard-cell-data'})

id = []

for i in range(len(pokemon_id)):
    id.append(pokemon_id[i].text)
    id[i] = int(id[i])

id = list(set(id))      # Sắp xếp và xóa các id bị trùng

In [ ]:
# Lấy danh sách urls cho web scrapping
urls = []
n = len(id)

for i in range(n):
    urls.append('https://pokemondb.net/pokedex/' + str(id[i]))

In [ ]:
# Thu thập dữ liệu nào
def get_pokemon_info(soup):
    # name
    name = soup.find('h1').text

    # types
    types_list = soup.find_all('td')[1].text.replace('\n', '')
    types_list = types_list[:-1].replace(' ', ' & ')        # type1 & type2

    # species
    species = soup.find_all('td')[2].text

    # height
    height = soup.find_all('td')[3].text.replace('\xa0', '')

    # weight
    weight = soup.find_all('td')[4].text.replace('\xa0', '')

    # bility & hidden Ability
    ability = soup.find_all('span', {'class': 'text-muted'})[0].text.replace('1. ', '')
    hid_ability = soup.find_all('small', {'class': 'text-muted'})[0].text.replace(' (hidden ability)', '')

    # catch_rate
    catch_rate = soup.find_all('td')[8].text.replace('\n', '')

    # base friendship 
    base_fs = soup.find_all('td')[9].text.replace('\n', '')

    # base exp
    base_exp = soup.find_all('td')[10].text.replace('\n', '')

    # growth_rate
    growth_rate = soup.find_all('td')[11].text.replace('\n', '')

    # genders
    genders =soup.find_all('td')[13].text.replace('\n', '')

    # base stats (base, min, max): health point, attack, defense, special attack, special defense, speed, total (base)
    hp = list(map(int, [soup.find_all('td')[15].text.replace('\n', ''),
                soup.find_all('td')[17].text.replace('\n', ''),
                soup.find_all('td')[18].text.replace('\n', '')]))

    atk = list(map(int, [soup.find_all('td')[19].text.replace('\n', ''),
                        soup.find_all('td')[21].text.replace('\n', ''),
                        soup.find_all('td')[22].text.replace('\n', '')]))

    defn = list(map(int, [soup.find_all('td')[23].text.replace('\n', ''),
                        soup.find_all('td')[25].text.replace('\n', ''),
                        soup.find_all('td')[26].text.replace('\n', '')]))

    sp_atk = list(map(int, [soup.find_all('td')[27].text.replace('\n', ''),
                            soup.find_all('td')[29].text.replace('\n', ''),
                            soup.find_all('td')[30].text.replace('\n', '')]))
    
    sp_def = list(map(int, [soup.find_all('td')[31].text.replace('\n', ''),
                            soup.find_all('td')[33].text.replace('\n', ''),
                            soup.find_all('td')[33].text.replace('\n', '')]))

    spd =  list(map(int, [soup.find_all('td')[35].text.replace('\n', ''),
                        soup.find_all('td')[37].text.replace('\n', ''),
                        soup.find_all('td')[38].text.replace('\n', '')]))

    total = int(soup.find_all('td')[39].text)

    return name, types_list, species, height, weight, ability, hid_ability, catch_rate, base_fs, base_exp, growth_rate, genders, hp, atk, defn, sp_atk, sp_def, spd, total

In [ ]:
data = []
for url in urls:
    source = requests.get(url).text
    soup = bs4.BeautifulSoup(source,'html.parser')
    data.append(get_pokemon_info(soup))

pokedex = pd.DataFrame(data = data)

In [57]:
pokedex

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Bulbasaur,Grass & Poison,Seed Pokémon,0.7m (2′04″),6.9kg (15.2lbs),Overgrow,Chlorophyll,"45 (5.9% with PokéBall, full HP)",50 (normal),64,Medium Slow,"87.5% male, 12.5% female","[45, 200, 294]","[49, 92, 216]","[49, 92, 216]","[65, 121, 251]","[65, 121, 121]","[45, 85, 207]",318
1,Ivysaur,Grass & Poison,Seed Pokémon,1.0m (3′03″),13.0kg (28.7lbs),Overgrow,Chlorophyll,"45 (5.9% with PokéBall, full HP)",50 (normal),142,Medium Slow,"87.5% male, 12.5% female","[60, 230, 324]","[62, 116, 245]","[63, 117, 247]","[80, 148, 284]","[80, 148, 148]","[60, 112, 240]",405
2,Venusaur,Grass & Poison,Seed Pokémon,2.0m (6′07″),100.0kg (220.5lbs),Overgrow,Chlorophyll,"45 (5.9% with PokéBall, full HP)",50 (normal),236,Medium Slow,"87.5% male, 12.5% female","[80, 270, 364]","[82, 152, 289]","[83, 153, 291]","[100, 184, 328]","[100, 184, 184]","[80, 148, 284]",525
3,Charmander,Fire,Lizard Pokémon,0.6m (2′00″),8.5kg (18.7lbs),Blaze,Solar Power,"45 (5.9% with PokéBall, full HP)",50 (normal),62,Medium Slow,"87.5% male, 12.5% female","[39, 188, 282]","[52, 98, 223]","[43, 81, 203]","[60, 112, 240]","[50, 94, 94]","[65, 121, 251]",309
4,Charmeleon,Fire,Flame Pokémon,1.1m (3′07″),19.0kg (41.9lbs),Blaze,Solar Power,"45 (5.9% with PokéBall, full HP)",50 (normal),142,Medium Slow,"87.5% male, 12.5% female","[58, 226, 320]","[64, 119, 249]","[58, 108, 236]","[80, 148, 284]","[65, 121, 121]","[80, 148, 284]",405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,Ursaluna,Ground & Normal,Peat Pokémon,2.4m (7′10″),290.0kg (639.3lbs),Guts,Unnerve,"20 (2.6% with PokéBall, full HP)",50 (normal),—,—,"50% male, 50% female","[130, 370, 464]","[140, 256, 416]","[105, 193, 339]","[45, 85, 207]","[80, 148, 148]","[50, 94, 218]",550
901,Basculegion,Water & Ghost,Big Fish Pokémon,3.0m (9′10″),110.0kg (242.5lbs),Rattled,Mold Breaker,"45 (5.9% with PokéBall, full HP)",50 (normal),—,—,"100% male, 0% female","[120, 350, 444]","[112, 206, 355]","[65, 121, 251]","[80, 148, 284]","[75, 139, 139]","[78, 144, 280]",530
902,Sneasler,Poison & Fighting,Free Climb Pokémon,1.3m (4′03″),43.0kg (94.8lbs),Pressure,Poison Touch,"20 (2.6% with PokéBall, full HP)",50 (normal),—,—,"50% male, 50% female","[80, 270, 364]","[130, 238, 394]","[60, 112, 240]","[40, 76, 196]","[80, 148, 148]","[120, 220, 372]",510
903,Overqwil,Dark & Poison,Pin Cluster Pokémon,2.5m (8′02″),60.5kg (133.4lbs),Poison Point,Intimidate,"45 (5.9% with PokéBall, full HP)",50 (normal),—,—,"50% male, 50% female","[85, 280, 374]","[115, 211, 361]","[95, 175, 317]","[65, 121, 251]","[65, 121, 121]","[85, 157, 295]",510
